In [1]:
# !yes|pip uninstall spacy
# !pip install spacy==3.1.1
# !python -m spacy info
import spacy
print(spacy.__version__)  # 3.1.1


3.5.2


In [2]:
#!yes | pip uninstall spacy

!python -m spacy info
import spacy
print(spacy.__version__)  # 3.1.1

2023-06-08 12:57:38.910386: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================== Info about spaCy ==============================

spaCy version    3.5.2                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-5.15.107+-x86_64-with-glibc2.31
Python version   3.10.11                       
Pipelines        en_core_web_sm (3.5.0)        

3.5.2


In [147]:
import spacy
import numpy as np
!yes|pip uninstall spacy-transformers
!pip install spacy-transformers

LANG = "ITA" # "ENG" or "ITA"

if LANG == "ITA":
  !python -m spacy download it_core_news_lg
  nlp = spacy.load("it_core_news_lg")
  import it_core_news_lg
  pass
elif LANG == "ENG":
  !python -m spacy download en_core_web_lg --force
  nlp = spacy.load("en_core_web_lg")
  import en_core_web_lg
  nlp = en_core_web_lg.load()
  pass




Found existing installation: spacy-transformers 1.2.4
Uninstalling spacy-transformers-1.2.4:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/spacy_transformers-1.2.4.dist-info/*
    /usr/local/lib/python3.10/dist-packages/spacy_transformers/*
Proceed (Y/n)?   Successfully uninstalled spacy-transformers-1.2.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy_transformers-1.2.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (190 kB)


2023-06-08 15:02:42.438821: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.9/567.9 MB 2.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_lg')


In [4]:
import spacy
import spacy_transformers
import numpy as np

doc = nlp("Frase italiana.")

print([(w.text, w.pos_) for w in doc])
print(len(nlp.vocab))

[('Frase', 'NOUN'), ('italiana', 'ADJ'), ('.', 'PUNCT')]
383


In [148]:
import random

word1 = nlp('nero')
word2 = nlp('bianco')

similarity = word1.similarity(word2)
print(f'{word1} ~= {word2} {similarity}') # 0.835

print(len(nlp.vocab.strings))
list(sorted(random.choices(list(nlp.vocab.strings), k=10)))

nero ~= bianco 0.8355581480087845
681834


['4325',
 'Aprica',
 'Dubai',
 'LHI',
 'Montato',
 'artistique',
 'ballonzola',
 'fablier',
 'i)1',
 'schillaci']

In [11]:
from numba import jit

@jit(nopython=True)
def cosine_similarity_numba(u:np.ndarray, v:np.ndarray):
    assert(u.shape[0] == v.shape[0])
    uv = 0
    uu = 0
    vv = 0
    for i in range(u.shape[0]):
        uv += u[i]*v[i]
        uu += u[i]*u[i]
        vv += v[i]*v[i]
    cos_theta = 1
    if uu != 0 and vv != 0:
        cos_theta = uv/np.sqrt(uu*vv)
    return cos_theta

In [47]:
def most_similar(word, topn=5, reverse=True):
  word = nlp.vocab[str(word)]
  queries = [
      w for w in nlp.vocab 
      if np.count_nonzero(w.vector)
  ]

  by_similarity = sorted(queries, key=lambda w: word.similarity(w), reverse=reverse)
  return [(w.lower_,w.similarity(word)) for w in by_similarity[:topn+1] if w.lower_ != word.lower_]

similar = most_similar("salume", topn=3)
print(similar)


dir(nlp.vocab["king"])
nlp.vocab["king"].has_vector
nlp

[('caserecci', 1.0), ('norcini', 1.0), ('salumieri', 1.0), ('casarecci', 1.0)]


In [92]:
from scipy.spatial import distance

In [120]:
def most_similar2(word, topn=5, distanceFunction=None, reverse=True):
  word = nlp.vocab[str(word)]
  queries = [
      w for w in nlp.vocab 
      if np.count_nonzero(w.vector)
  ]

  if distanceFunction is None:
    distanceFunction = cosine_similarity_numba

  by_similarity = sorted(queries, key=lambda w: distanceFunction(word.vector, w.vector), reverse=reverse)
  return [(w.lower_,w.similarity(word)) for w in by_similarity[:topn+1] if w.lower_ != word.lower_]

similar = most_similar2("salume", distanceFunction=distance.russellrao, reverse=False)
print(similar)


dir(nlp.vocab["king"])
nlp.vocab["king"].has_vector
nlp

[('mosti', 0.6962182521820068), ('spumanti', 0.6962182521820068), ('enoici', 0.6962182521820068), ('crus', 0.6962182521820068), ('vini', 0.6962182521820068), ('vignaioli', 0.6962182521820068)]


In [78]:
from numpy import dot
from numpy.linalg import norm

# cosine similarity
cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))

def most_similar3(word, topn=5, reverse=True):
  word = nlp.vocab[str(word)]
  queries = [
      w for w in nlp.vocab 
      if np.count_nonzero(w.vector)
  ]

  by_similarity = sorted(queries, key=lambda w: cosine(word.vector, w.vector), reverse=reverse)
  return [(w.lower_,w.similarity(word)) for w in by_similarity[:topn+1] if w.lower_ != word.lower_]

similar = most_similar3("salume", topn=3)
print(similar)


dir(nlp.vocab["king"])
nlp.vocab["king"].has_vector
nlp

[('caserecci', 1.0), ('norcini', 1.0), ('salumieri', 1.0), ('casarecci', 1.0)]


In [15]:
# constansts

START_WORD_LENGTHS = [4,5,6]

MIN_WORD_LENGTH = 3
MAX_WORD_LENGTH = 9

NUM_STEPS = 8
MIN_STEPS = 5



In [16]:
'''
Created on Feb 7, 2013

@author: olegs
'''

ROMAN_CONSTANTS = (
            ( "", "I", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX" ),
            ( "", "X", "XX", "XXX", "XL", "L", "LX", "LXX", "LXXX", "XC" ),
            ( "", "C", "CC", "CCC", "CD", "D", "DC", "DCC", "DCCC", "CM" ),
            ( "", "M", "MM", "MMM", "",   "",  "-",  "",    "",     ""   ),
        )

ROMAN_SYMBOL_MAP = dict(I=1, V=5, X=10, L=50, C=100, D=500, M=1000)

CUTOFF = 4000
BIG_DEC = 2900
BIG_ROMAN = "MMCM"
ROMAN_NOUGHT = "nulla"

def digits(num):
    if num < 0:
        raise Exception('range error: negative numbers not supported')
    if num % 1 != 0.0:
        raise Exception('floating point numbers not supported')
    res = []
    while num > 0:
        res.append(num % 10)
        num //= 10
    return res

def toString(num, emptyZero=False):
    if num < CUTOFF:
        digitlist = digits(num)
        if digitlist:
            res = reversed([ ROMAN_CONSTANTS[order][digit] for order, digit in enumerate(digitlist) ])
            return "".join(res)
        else:
            return "" if emptyZero else ROMAN_NOUGHT 
    else:
        if num % 1 != 0.0:
            raise Exception('floating point numbers not supported')
        # For numbers over or equal the CUTOFF, the remainder of division by 2900
        # is represented as above, prepended with the multiples of MMCM (2900 in Roman),
        # which guarantees no more than 3 repetitive Ms.
        return BIG_ROMAN * (num // BIG_DEC) + toString(num % BIG_DEC, emptyZero=True)

def parse(numeral):
    numeral = numeral.upper()
    result = 0

    lastVal = 0
    lastCount = 0
    subtraction = False
    for symbol in numeral[::-1]:
        value = ROMAN_SYMBOL_MAP.get(symbol)
        if not value:
            return False
        if lastVal == 0:
            lastCount = 1
            lastVal = value
        elif lastVal == value:
            lastCount += 1
            # exceptions
        else:
            result += (-1 if subtraction else 1) * lastVal * lastCount
            subtraction = lastVal > value
            lastCount = 1
            lastVal = value
    
    result = result + (-1 if subtraction else 1) * lastVal * lastCount
    return True

In [17]:
import requests
import time

dictionaryFileName = 'assets/dictionary_FULL_IT.txt'
dictionaryFileName = 'assets/Ita.txt'

    
def normalizeWord(word):
    return word.strip().upper()
    
LOAD_FROM_URL = True

t0 = time.time()

lines = []
if LOAD_FROM_URL:
    baseUrl = 'https://azrafe7.github.io/'
    f = requests.get(baseUrl + dictionaryFileName)
    lines = f.text.splitlines()
else:
    with open(dictionaryFileName) as f:
        lines = f.read().splitlines()

filteredWords = []
for line in lines:
    word = normalizeWord(line)
    length = len(word)
    if length >= MIN_WORD_LENGTH and length <= MAX_WORD_LENGTH and not parse(word):
        filteredWords.append(word)

print("Dictionary loaded in {0:.2f}s".format(time.time() - t0))

print('{0}/{1}'.format(len(filteredWords), len(lines)))

print(filteredWords[:10])

Dictionary loaded in 0.87s
401123/710993
['AALENIANA', 'AALENIANE', 'AALENIANI', 'AALENIANO', 'ABABUA', 'ABACA', 'ABACETI', 'ABACETO', 'ABACHI', 'ABACHISTA']


In [151]:
for w in filteredWords:
  nlp.vocab[w.lower()]

In [159]:
import time

word = 'scrofa'

word2 = 'maiale'
print(nlp.vocab[word].similarity(nlp.vocab[word2]))
print(cosine_similarity_numba(nlp.vocab[word].vector, nlp.vocab[word2].vector))

# t0 = time.time()
# similars = most_similar(word
# print('{0:.2f}s'.format(time.time() - t0))
# display(similars)

t0 = time.time()
similars2 = most_similar2(word, topn=30, distanceFunction=None, reverse=True)
print('{0:.2f}s'.format(time.time() - t0))
display(similars2)

t0 = time.time()
similars2 = most_similar2(word, topn=30, distanceFunction=distance.chebyshev, reverse=False)
print('{0:.2f}s'.format(time.time() - t0))
display(similars2)

# t0 = time.time()
# similars3 = most_similar3(word, topn=10, reverse=True)
# print('{0:.2f}s'.format(time.time() - t0))
# display(similars3)

0.5014864802360535
0.5014864654258369
2.39s


[('scrofe', 0.6642468571662903),
 ('vacca', 0.6385034918785095),
 ('mucca', 0.6244244575500488),
 ('cerva', 0.6015369296073914),
 ('pecora', 0.5831999778747559),
 ('capra', 0.5800632834434509),
 ('lepre', 0.5788684487342834),
 ('lupa', 0.5679877400398254),
 ('cervo', 0.5673580765724182),
 ('gallina', 0.5636082291603088),
 ('maialina', 0.558182954788208),
 ('iena', 0.557919979095459),
 ('capretta', 0.5569721460342407),
 ('cagna', 0.555469810962677),
 ('mandria', 0.5463758707046509),
 ('cervone', 0.5436885356903076),
 ('bovidi', 0.5420840978622437),
 ('bestiola', 0.5411077737808228),
 ('scimmia', 0.5401797890663147),
 ('ovaiola', 0.5400537848472595),
 ('bestia', 0.5387506484985352),
 ('scrofola', 0.5337382555007935),
 ('sbrana', 0.5306189656257629),
 ('giovenca', 0.5292507410049438),
 ('femmina', 0.5281515717506409),
 ('vitellina', 0.5271637439727783),
 ('bruca', 0.5271130800247192),
 ('zanna', 0.5220664739608765),
 ('gatta', 0.5218215584754944),
 ('capriolo', 0.5217816233634949)]

6.93s


[('babbuino', 0.5042772889137268),
 ('scrofe', 0.6642468571662903),
 ('miagola', 0.42082837224006653),
 ('cagnaccio', 0.43688228726387024),
 ('azzanna', 0.3994207978248596),
 ('puledra', 0.5144197344779968),
 ('pollastra', 0.48318392038345337),
 ('scimmia', 0.5401797890663147),
 ('capriolo', 0.5217816233634949),
 ('trisavola', 0.29908379912376404),
 ('issopo', 0.19020125269889832),
 ('onagro', 0.2515365779399872),
 ('lupoide', 0.4403741955757141),
 ('trisnonna', 0.30644306540489197),
 ('scrofola', 0.5337382555007935),
 ('o_o', 0.07505510002374649),
 ('vitellone', 0.4390723407268524),
 ('strabuzza', 0.20887790620326996),
 ('cediglia', 0.2527424693107605),
 ('guaito', 0.2804921865463257),
 ('scudiscio', 0.2600635290145874),
 ('mufloni', 0.413777619600296),
 ('cagnacci', 0.3986813724040985),
 ('porcilaia', 0.4753342270851135),
 ('abomaso', 0.3224746584892273),
 ('buffalo', 0.36930644512176514),
 ('bicefala', 0.3548136055469513),
 ('forzuto', 0.2843242883682251),
 ('miagolare', 0.334054410

In [149]:
'-ci' in nlp.vocab
#[x.text for x in nlp.vocab]

False

In [190]:
word = 'cotto'
lexeme = nlp.vocab[word]
lexeme.orth_
print(lexeme.prob)
lexeme.sentiment
doc = nlp(word)
doc
token = doc[0]
token
token.sentiment
print(token.lemma_)
print(token.pos_)
print(token.morph.to_dict())
print(token.morph.get('Gender'))


-20.0
cotto
VERB
{'Gender': 'Masc', 'Number': 'Sing', 'Tense': 'Past', 'VerbForm': 'Part'}
['Masc']


In [ ]:
w = nlp.vocab['chaos']
sorted([x.text for x in nlp.vocab])
len([x.text for x in nlp.vocab])


In [ ]:
import spacy

from numpy import dot
from numpy.linalg import norm

parser = spacy.load("en_core_web_md")


In [153]:
from numpy import dot
from numpy.linalg import norm

word = u"dog"
nasa = nlp.vocab[word]

# cosine similarity
cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))

# gather all known words, take only the lowercased versions
allWords = list({w for w in nlp.vocab if w.has_vector and w.orth_.islower() and w.is_alpha and w.lower_ != word})

# sort by similarity to NASA
allWords.sort(key=lambda w: cosine(w.vector, nasa.vector))
allWords.reverse()
print(f"Top 10 most similar words to {word}:")
for word in allWords[:20]:
    print(word.orth_, cosine(nasa.vector, word.vector))

#[w.text for w in allWords]

Top 10 most similar words to dog:
dogs 0.7097339
baby 0.5822533
chicken 0.53386366
spots 0.51317126
bear 0.5050963
bull 0.49388632
sled 0.4913776
burger 0.48703855
burgers 0.47693646
hamburger 0.4680303
pants 0.46468768
cocker 0.4646298
beach 0.44747853
pony 0.4451662
pudding 0.44414398
teddy 0.43525335
beagle 0.4318026
sugar 0.4293324
biking 0.41360223
truck 0.4121126


In [157]:
import random

def replace_synonyms(text):
    doc = nlp(text)
    new_doc = []
    for token in doc:
        print(token.has_vector, token.pos_)
        if token.has_vector and token.pos_ in ["NOUN", "VERB", "ADJ", "ADV", "PROPN"]:
            synonyms = [t for t in nlp.vocab if t.has_vector and t.similarity(token) > 0.8 and t.text != token.text]
            if synonyms:
                print(token.text, [s.text for s in synonyms])
                new_token = random.choice(synonyms)
                new_doc.append(new_token.text)
            else:
                new_doc.append(token.text)
        else:
            new_doc.append(token.text)
    return " ".join(new_doc)

text = "bere una birra in compagnia"
synonym_replaced_text = replace_synonyms(text)
print(synonym_replaced_text)


True VERB
True DET
True NOUN
birra ['birr']
True ADP
True NOUN
bere una birr in compagnia
